# Kaggle Project 6 - Santander Value Prediction Challenge

## Prepared by: Dwaipayan Mukherjee (2211569)

### Necessary Imports:

In [1]:
import numpy as np
import pandas as pd
import gc

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


### Loading train and test set:

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### Viewing the train and test data:

In [3]:
# Viewing the train data
train_df.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [4]:
# Viewing the test data
test_df.head()

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Checking the info of the data:

In [5]:
# train data
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [6]:
# test data
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 4992 entries, ID to 9fc776466
dtypes: float64(4991), object(1)
memory usage: 1.8+ GB


### Checking for Null values:

In [7]:
# train data
print("Total Train Features with NaN Values = " + str(train_df.columns[train_df.isnull().sum() != 0].size))
if (train_df.columns[train_df.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(train_df.columns[train_df.isnull().sum() != 0])))
    train_df[train_df.columns[train_df.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total Train Features with NaN Values = 0


In [8]:
# test data
print("Total Test Features with NaN Values = " + str(test_df.columns[test_df.isnull().sum() != 0].size))
if (test_df.columns[test_df.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(test_df.columns[test_df.isnull().sum() != 0])))
    test_df[test_df.columns[test_df.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total Test Features with NaN Values = 0


### Checking and removing constant attribute columns:

In [9]:

colsToRemove = []
for col in train_df.columns:
    if col !='ID' and col !='target':
        if train_df[col].std() == 0:
            colsToRemove.append(col)

train_df.drop(colsToRemove, axis=1, inplace=True)

test_df.drop(colsToRemove, axis=1, inplace=True)

print("Deleted `{}` constant columns\n".format(len(colsToRemove)))
print(colsToRemove)

Deleted `256` constant columns

['d5308d8bc', 'c330f1a67', 'eeac16933', '7df8788e8', '5b91580ee', '6f29fbbc7', '46dafc868', 'ae41a98b6', 'f416800e9', '6d07828ca', '7ac332a1d', '70ee7950a', '833b35a7c', '2f9969eab', '8b1372217', '68322788b', '2288ac1a6', 'dc7f76962', '467044c26', '39ebfbfd9', '9a5ff8c23', 'f6fac27c8', '664e2800e', 'ae28689a2', 'd87dcac58', '4065efbb6', 'f944d9d43', 'c2c4491d5', 'a4346e2e2', '1af366d4f', 'cfff5b7c8', 'da215e99e', '5acd26139', '9be9c6cef', '1210d0271', '21b0a54cb', 'da35e792b', '754c502dd', '0b346adbd', '0f196b049', 'b603ed95d', '2a50e001c', '1e81432e7', '10350ea43', '3c7c7e24c', '7585fce2a', '64d036163', 'f25d9935c', 'd98484125', '95c85e227', '9a5273600', '746cdb817', '6377a6293', '7d944fb0c', '87eb21c50', '5ea313a8c', '0987a65a1', '2fb7c2443', 'f5dde409b', '1ae50d4c3', '2b21cd7d8', '0db8a9272', '804d8b55b', '76f135fa6', '7d7182143', 'f88e61ae6', '378ed28e0', 'ca4ba131e', '1352ddae5', '2b601ad67', '6e42ff7c7', '22196a84c', '0e410eb3d', '992e6d1d3', '90a7

### Removing duplicate columns:

In [10]:
%%time
# Function to find duplicate columns
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cs = frame[v].columns
        vs = frame[v]
        lcs = len(cs)

        for i in range(lcs):
            ia = vs.iloc[:,i].values
            for j in range(i+1, lcs):
                ja = vs.iloc[:,j].values
                if np.array_equal(ia, ja):
                    dups.append(cs[i])
                    break

    return dups

colsToRemove = duplicate_columns(train_df)
print(colsToRemove)

['34ceb0081', '8d57e2749', '168b3e5bc', 'a765da8bc', 'acc5b709d']
Wall time: 12min 20s


In [11]:
# Dropping duplicate columns from train data:
train_df.T.drop_duplicates(keep='last').T

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,3.8e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000fbd867,600000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0027d6b71,1e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0028cbf45,2e+06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,002a68644,1.44e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4454,ff85154c8,1.065e+06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,ffb6b3f4f,48000,0,0,0,0,0,0,0,0,...,0,0,80000,0,0,0,0,0,0,0
4456,ffcf61eb6,2.8e+06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4457,ffea67e98,1e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Removing duplicate columns from train data
train_df.drop(colsToRemove, axis=1, inplace=True) 

# Removing duplicate columns from test data
test_df.drop(colsToRemove, axis=1, inplace=True)

print("Removed `{}` Duplicate Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Removed `5` Duplicate Columns

['34ceb0081', '8d57e2749', '168b3e5bc', 'a765da8bc', 'acc5b709d']


### Removing sparse data:

In [13]:
# Function to remove sparse columns
def drop_sparse(train, test):
    flist = [x for x in train.columns if not x in ['ID','target']]
    for f in flist:
        if len(np.unique(train[f]))<2:
            train.drop(f, axis=1, inplace=True)
            test.drop(f, axis=1, inplace=True)
    return train, test

In [14]:
%%time
train_df, test_df = drop_sparse(train_df, test_df)

Wall time: 1.68 s


In [15]:
gc.collect()
print("Train set size: {}".format(train_df.shape))
print("Test set size: {}".format(test_df.shape))

Train set size: (4459, 4732)
Test set size: (49342, 4731)


In [16]:
# Creating training and test data for modeling:
X_train = train_df.drop(["ID", "target"], axis=1)
y_train = np.log1p(train_df["target"].values)

X_test = test_df.drop(["ID"], axis=1)

In [17]:
# train test split of the train data:
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [18]:
# Defining a function for LightGBM model:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.004,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=150, 
                      evals_result=evals_result)
    
    pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

In [19]:
%%time
# Training the LightGBM model:
pred_test_lgb, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed...")

[LightGBM] [Warning] Unknown parameter: bagging_frequency
Training until validation scores don't improve for 100 rounds
[150]	training's rmse: 1.50845	valid_1's rmse: 1.539
[300]	training's rmse: 1.3446	valid_1's rmse: 1.46591
[450]	training's rmse: 1.23333	valid_1's rmse: 1.43454
[600]	training's rmse: 1.15002	valid_1's rmse: 1.42157
[750]	training's rmse: 1.08396	valid_1's rmse: 1.41615
[900]	training's rmse: 1.03041	valid_1's rmse: 1.41411
Early stopping, best iteration is:
[877]	training's rmse: 1.03806	valid_1's rmse: 1.4137
LightGBM Training Completed...
Wall time: 1min 5s


In [20]:
# Feature Importance
print("Features Importance...")
gain = model.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model.feature_name(), 
                   'split':model.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:50])

Features Importance...
        feature  split      gain
4130  f190486d6    795  9.217105
2375  58e2e02e6    703  5.445323
3465  eeb9cd3aa    662  4.520153
4020  15ace8c9f    514  3.023536
2614  9fd594eec    362  2.933775
8     20aa07010    425  2.229353
3571  58232a6fb    371  1.503661
834   6eef030c1    313  1.411549
1457  b43a7cfd5    385  1.285384
3661  491b9ee45    276  1.066377
2687  fb0f5dbfe    408  1.065763
1482  024c577b9    260  0.936856
2079  58e056e12    338  0.904839
4508  c47340d97    320  0.878493
4343  1702b5bf0    282  0.873053
3867  2288333b4    174  0.867616
4185  f74e8f13d    338  0.832932
566   66ace2992    280  0.794802
828   6786ea46d    175  0.771087
3722  d6bb78916    286  0.767982
3791  ed8ff54b5    166  0.763305
3220  ced6a7e91    247  0.758728
4028  5c6487af1    193  0.704117
1378  6cf7866c1    162  0.696926
863   fc99f9426    235  0.679075
3886  50e4f96cf    140  0.666640
34    87ffda550    163  0.623059
213   186b87c05     68  0.618299
853   bc70cbc26    1

In [21]:
# Function to train a XGBoost Model:
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {'objective': 'reg:linear', 
          'eval_metric': 'rmse',
          'eta': 0.001,
          'max_depth': 10, 
          'subsample': 0.6, 
          'colsample_bytree': 0.6,
          'alpha':0.001,
          'random_state': 42, 
          'silent': True}
    
    tr_data = xgb.DMatrix(train_X, train_y)
    va_data = xgb.DMatrix(val_X, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 100, verbose_eval=100)
    
    dtest = xgb.DMatrix(test_X)
    xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return xgb_pred_y, model_xgb

In [22]:
%%time
# Training the XGBoost model:
pred_test_xgb, model_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
print("XGB Training Completed...")

[11:43:05] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[11:43:05] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-rmse:14.08764	valid-rmse:14.07678
[100]	train-rmse:12.76875	valid-rmse:12.75679
[200]	train-rmse:11.57719	valid-rmse:11.56388
[300]	train-rmse:10.50042	valid-rmse:10.48590
[400]	train-rmse:9.52826	valid-rmse:9.51335
[500]	train-rmse:8.65075	valid-rmse:8.63628
[600]	train-rmse:7.85853	valid-rmse:7.84507
[700]	train-rmse:7.14378	valid-rmse:7.13204
[800]	train-rmse:6.49866	valid-rmse:6.48963
[900]	train-rmse:5.91700	valid-rmse:5.91129
[1000]	train-rmse:5.39177	valid-rmse:5.39058
[1100]	train-rmse:4.91908	valid-rmse:4.92353
[1200]	train-rmse:4.49300	valid-rmse:

In [23]:
%%time
# Training a CatBoost model:
cb_model = CatBoostRegressor(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
cb_model.fit(dev_X, dev_y,
             eval_set=(val_X, val_y),
             use_best_model=True,
             verbose=50)

0:	learn: 1.7518683	test: 1.6878429	best: 1.6878429 (0)	total: 3.65s	remaining: 30m 20s
50:	learn: 1.4789181	test: 1.5197196	best: 1.5197196 (50)	total: 3m 13s	remaining: 28m 26s
100:	learn: 1.3788775	test: 1.4780503	best: 1.4780503 (100)	total: 6m 19s	remaining: 24m 59s
150:	learn: 1.3203081	test: 1.4647232	best: 1.4647232 (150)	total: 9m 19s	remaining: 21m 32s
200:	learn: 1.2546056	test: 1.4505032	best: 1.4504536 (198)	total: 12m 28s	remaining: 18m 33s
250:	learn: 1.1817435	test: 1.4381513	best: 1.4381513 (250)	total: 15m 26s	remaining: 15m 18s
300:	learn: 1.1208241	test: 1.4320748	best: 1.4314980 (297)	total: 18m 31s	remaining: 12m 15s
350:	learn: 1.0770301	test: 1.4298988	best: 1.4297384 (347)	total: 21m 29s	remaining: 9m 7s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.428228031
bestIteration = 376

Shrink model to first 377 iterations.
Wall time: 24min 15s


In [24]:
pred_test_cat = np.expm1(cb_model.predict(X_test))

In [26]:
# Fusing the three models:
sub = pd.read_csv('sample_submission.csv')

sub_lgb = pd.DataFrame()
sub_lgb["target"] = pred_test_lgb

sub_xgb = pd.DataFrame()
sub_xgb["target"] = pred_test_xgb

sub_cat = pd.DataFrame()
sub_cat["target"] = pred_test_cat

sub["target"] = (sub_lgb["target"] * 0.5 + sub_xgb["target"] * 0.3 + sub_cat["target"] * 0.2)

In [ ]:
print(sub.head())
sub.to_csv('sub_lgb_xgb_cat.csv', index=False)
sub.to_csv('submission.csv', index=False)